# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --db uk_food --collection establishments --file establishments.json --jsonArray`

In [183]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [184]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [185]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'classDB', 'config', 'epa', 'local', 'met', 'petsitly_marketing', 'travel_db', 'uk_food']


In [186]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [187]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [188]:
# assign the collection to a variable
establishments = db['establishments']

In [189]:
# review a document in the establishments collection
document_count = establishments.count_documents({})
print(document_count)

39779


In [190]:
#Find and display one document in the establishments collection using find_one and display with pprint.
one_document = establishments.find_one()
pprint(one_document)

{'AddressLine1': '1 The Droveway',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'FirstLight Bar & Café',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.64041272672,
 'FHRSID': 1290215,
 'LocalAuthorityBusinessID': 'PI/000083029',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DH',
 'RatingDate': '2020-11-09T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('650a560dce67469186b7f31b'),
 'geocode': {'latitude': '51.150625', 'longitude': '1.378231'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1290215',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [191]:
# Create a dictionary for the new restaurant data
new_restaurant_data ={
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [192]:
# Query to find and remove existing documents with the same BusinessName
existing_restaurant_query = {"BusinessName": new_restaurant_data["BusinessName"]}
establishments.delete_many(existing_restaurant_query)

In [193]:
# Insert the new restaurant into the collection
new_restaurant = establishments.insert_one(new_restaurant_data)

In [194]:
# Check that the new restaurant was inserted
print(new_restaurant)

In [195]:
# Confirm the document count after the insertion
document_count = establishments.count_documents({})
print(document_count)

39780


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [196]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
# query to find the business "Restaurant/Cafe/Canteen"
business_type = {"BusinessType": "Restaurant/Cafe/Canteen"}

# fields to include in the result
business_type_fields = {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}

# results combining the type of business and ID
result = establishments.find_one(business_type, business_type_fields)

# Print the result
pprint(result)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [197]:
# Update the new restaurant with the correct BusinessTypeID
# find the new restaurant by BusinessName
new_restaurant_name = {"BusinessName": "Penang Flavours"}

# update with the correct BusinessTypeID
new_restaurant_business_type_ID = {"$set": {"BusinessTypeID": 1}}

# Update the document with the correct BusinessTypeID
update_business_type_ID = establishments.update_one(new_restaurant_name, new_restaurant_business_type_ID)

In [198]:
# Confirm that the new restaurant was updated
updated_restaurant = establishments.find_one({"BusinessName": "Penang Flavours"})
# Print the business name and new BusinessTypeID
business_name = updated_restaurant.get("BusinessName", "N/A")
business_type_id = updated_restaurant.get("BusinessTypeID", "N/A")

print(f"Business Name: {business_name}")
print(f"New Business Type ID: {business_type_id}")

Business Name: Penang Flavours
New Business Type ID: 1


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [199]:
# Find how many documents have LocalAuthorityName as "Dover"
#find the documents in 'Dover'
dover_query = {"LocalAuthorityName": "Dover"}

# count the number of documents in 'Dover'
dover_count = establishments.count_documents(dover_query)
print(dover_count)

994


In [200]:
# Delete all documents where LocalAuthorityName is "Dover"
# query to match documents in 'Dover'
dover_query = {"LocalAuthorityName": "Dover"}

# delete documents in 'Dover'
delete_result = establishments.delete_many(dover_query)

# Print the number of documents deleted
print(delete_result.deleted_count)

994


In [201]:
# Check if any remaining documents include Dover
dover_count = establishments.count_documents(dover_query)
print(dover_count)

0


In [202]:
# Check that other documents remain with 'find_one'
one_document = establishments.find_one()
pprint(one_document)

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('650a560dce67469186b7f601'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': 

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [203]:
# Change the data type from String to Decimal for longitude and latitude
update_data_type = establishments.update_many({}, [
    {"$set": {"geocode.longitude": {"$toDouble": "$geocode.longitude"}}},
    {"$set": {"geocode.latitude": {"$toDouble": "$geocode.latitude"}}}
])

#check if the data type was changed
one_document = establishments.find_one()

# Check and print the data types of the "longitude" and "latitude"
geocode = one_document["geocode"]
longitude_type = type(geocode.get("longitude"))
latitude_type = type(geocode.get("latitude"))
print(f"Longitude Data Type: {longitude_type}")
print(f"Latitude Data Type: {latitude_type}")

Longitude Data Type: <class 'float'>
Latitude Data Type: <class 'float'>


Use `update_many` to convert `RatingValue` to integer numbers.

In [204]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [{'$set': {"RatingValue": ""}}])

In [206]:
# Change the data type from String to Integer for RatingValue
establishments.update_many(
    {"RatingValue": {"$ne": ""}},  # Filter out empty strings
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]
)

In [207]:
# Check that the coordinates and rating value are now numbers
one_document = establishments.find_one()
rating_value_type = type(one_document.get("RatingValue"))
print(rating_value_type)

<class 'int'>
